<img align="right" style="max-width: 200px; height: auto" src="https://github.com/HSG-AIML-Teaching/MLBBWL-2022FS/blob/main/lab_02/hsg_logo.png?raw=1">

#  Lab 2 - "Unsupervised Learning with Scikit-Learn"

Machine Learning (BBWL), University of St. Gallen, Spring Term 2023, Michael Mommert

In our previous lecture, we discussed different unsupervised learning methods with a focus on **clustering** and **dimensionality reduction. Today, we will apply some of these methods to different data modalities.

## Lab Objectives:
* comparing different clustering methods for our Iris data set
* clustering applied to image data
* dimensionality reduction

## Clustering the Iris data set

We revisit our famous Iris data set:

In [ ]:
import numpy as np
from sklearn.datasets import load_iris

data = load_iris()
x = data.data
y = data.target
x.shape, y.shape

As a quick reminder: the data set (`x`) contains measures features of 150 iris flowers. The features are petal length, petal width, sepal length and sepal width. For each speciment we also know the class of flower it has been attributed to by a botanist (`y`).

Let's plot some two-dimensional projection of the four-dimensional data set:

In [ ]:
import matplotlib.pyplot as plt

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.scatter(x[:,0], x[:,1])
ax1.set_xlabel(data.feature_names[0])
ax1.set_ylabel(data.feature_names[1])

ax2.scatter(x[:,2], x[:,3])
ax2.set_xlabel(data.feature_names[2])
ax2.set_ylabel(data.feature_names[3])

The plots clearly show structure in the data: if we ignore the labels (since we are doing unsupervised learning here), there seem to be 2 or three clusters in the data. 

Of course, we do know the ground-truth labels for each of those data points. If we take advantage of these labels, the plot would look like this, exposing the different clusters based on their different class affiliations:

In [ ]:
import matplotlib.pyplot as plt

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.scatter(x[:,0], x[:,1], c=y)
ax1.set_xlabel(data.feature_names[0])
ax1.set_ylabel(data.feature_names[1])

ax2.scatter(x[:,2], x[:,3], c=y)
ax2.set_xlabel(data.feature_names[2])
ax2.set_ylabel(data.feature_names[3])

Now let's use some clustering methods to see how well we can identify these clusters.

### K-means


We start with a simple $k$-means clustering algorithm.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# scale the data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# train and apply k-Means
model = KMeans(n_clusters=2)
pred = model.fit_predict(x_scaled)


Please note how similar the `sklearn` API is to the supervised methods that we got to know two weeks ago. In fact, you might wonder why there is even a `predict` method, since we are only doing unsupervised learning.

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.scatter(x[:,0], x[:,1], c=pred, alpha=0.5)
ax1.set_xlabel(data.feature_names[0])
ax1.set_ylabel(data.feature_names[1])

ax2.scatter(x[:,2], x[:,3], c=pred, alpha=0.5)
ax2.set_xlabel(data.feature_names[2])
ax2.set_ylabel(data.feature_names[3])

The result looks good for $k=2$ - the two obvious clusters are easily separated.

**Question**: What about $k=3$?

What you should observe is that $k$-Means for $k=3$ indeed does a decent job at identifying the three clusters. Keep in mind that the goal here is not to replicate the class labels like in supervised learning. Instead, this method simply identifies agglomerations of data points and those often (but not always!) coincide with different classes of data points. 

## More complex clustering examples

### Moons data set

We will create a *moons* data set of two intertwined crescents:

In [ ]:
from sklearn.datasets import make_moons

# generate data set
x, y = make_moons(n_samples=1000, noise=0.05, random_state=42)
x = scaler.fit_transform(x) # scale the data set

f, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.scatter(x[:,0], x[:,1], color=[{0: 'red', 1: 'green', 2: 'blue'}[l] for l in y])


We already saw that $k$-Means will fail with this data set:

In [ ]:
model = KMeans(n_clusters=2)
pred = model.fit_predict(x)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.scatter(x[:,0], x[:,1], color=[{0: 'red', 1: 'green', 2: 'blue'}[l] for l in y])
ax1.set_title('Ground Truth')
ax2.scatter(x[:,0], x[:,1], color=[{0: 'red', 1: 'green', 2: 'blue'}[l] for l in pred])
ax2.set_title('Prediction')

**Question**: Why does $k$-Means fare badly in this case?

**Question**: What other clustering method might provide better results in this specific case? Can you implement it here?

In [ ]:
# we try hierarchical clustering
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=2)
pred = model.fit_predict(x)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.scatter(x[:,0], x[:,1], color=[{0: 'red', 1: 'green', 2: 'blue'}[l] for l in y])
ax1.set_title('Ground Truth')
ax2.scatter(x[:,0], x[:,1], color=[{0: 'red', 1: 'green', 2: 'blue'}[l] for l in pred])
ax2.set_title('Prediction')

# agglomerative clustering works a little bit better; but still not great

In [ ]:
# we try DBSCAN
from sklearn.cluster import DBSCAN

model = DBSCAN(eps=0.2, min_samples=3)
pred = model.fit_predict(x)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.scatter(x[:,0], x[:,1], color=[{0: 'red', 1: 'green', 2: 'blue'}[l%3] for l in y])
ax1.set_title('Ground Truth')
ax2.scatter(x[:,0], x[:,1], color=[{0: 'red', 1: 'green', 2: 'blue'}[l%3] for l in pred])
ax2.set_title('Prediction')

# after proper tuning, DBSCAN does a great job!

### Noisy data

In the following example we will try to reveal clusters in a noisy data set. *Noisy* refers here to the fact that in addition to the different clusters, the data set also contains a number of background data points that do not belong to either cluster. This situation is rather common in real life.

In [ ]:
from sklearn.datasets import make_blobs
from numpy.random import random
np.random.seed(42)

# generate data set
x, y = make_blobs(n_samples=500, n_features=2, random_state=42)
x_noise = random((200, 2))*(x.max()-x.min())+x.min()
x = np.vstack([x, x_noise])
y = np.hstack([y, [-1]*len(x_noise)])

f, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.scatter(x[:,0], x[:,1], color=[{-1:'gray', 0: 'red', 1: 'green', 2: 'blue'}[l] for l in y])


Let's check $k$-means, assuming that we know that there are 3 clusters:

In [ ]:
model = KMeans(n_clusters=3)
pred = model.fit_predict(x)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.scatter(x[:,0], x[:,1], color=[{-1: 'gray', 0: 'red', 1: 'green', 2: 'blue'}[l] for l in y])
ax1.set_title('Ground Truth')
ax2.scatter(x[:,0], x[:,1], color=[{-1: 'gray', 0: 'red', 1: 'green', 2: 'blue'}[l] for l in pred])
ax2.set_title('Prediction')

$k$-Means can identify the clusters properly, but it also picks up all the noise and assigns it to a cluster.


**Question**: Is there a clustering method that can deal with noise? If so, implement it!

In [ ]:
# we try DBSCAN
model = DBSCAN(eps=1, min_samples=10)
pred = model.fit_predict(x)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.scatter(x[:,0], x[:,1], color=[{-1: 'gray', 0: 'red', 1: 'green', 2: 'blue'}[-1 if l<0 else l%3] for l in y])
ax1.set_title('Ground Truth')
ax2.scatter(x[:,0], x[:,1], color=[{-1: 'gray', 0: 'red', 1: 'green', 2: 'blue'}[-1 if l<0 else l%3] for l in pred])
ax2.set_title('Prediction')

# after proper tuning, DBSCAN does a great job!

## Clustering of image data

We now saw plenty of examples for how to apply clustering methods to numerical data. What about image data? Can we cluster images?

Let's download some images of digits:

In [ ]:
from sklearn.datasets import load_digits

data = load_digits()

x = data.data
y = data.target
x.shape

What do they look like?

In [ ]:
x[0]

Hmm. That's a linearized image. Let's reshape the vector and plot the result:

In [ ]:
plt.imshow(x[0].reshape(8 ,8))

That's a zero, right?

In [ ]:
y[0]

Correct!

But can we simply apply clustering to the linearized image data? Let's try:

In [ ]:
model = KMeans(n_clusters=10)
pred = model.fit_predict(x)

Ok, but how do we check if it learned anything useful? We can average all image that belong to the same cluster and have a look at the averaged images:

In [ ]:
digits = np.empty((10, 64))
# create average images 
for i in range(10):
    digits[i] = np.average(x[pred == i], axis=0)

# reshape the vectors to 8x8 images
digits = digits.reshape(10, 8, 8)

f, ax = plt.subplots(2, 5, figsize=(10, 4), sharex=True, sharey=True)
ax = np.ravel(ax)

for i in range(10):
    ax[i].imshow(digits[i])
    ax[i].set_xticks([])
    ax[1].set_yticks([])

Those look like decent digits. Is it possible that we can cluster these vectorized images of digits with something as simple as $k$-Means?

Let's do a quick check: we will extract the ground-truth labels of our cluster members individually:

In [ ]:
y[pred == 4]

Naturally, the digits are off, since the cluster ids are assigned randomly. But it seems to be the case that most samples of the same cluster show the same digit.

Think about this for a minute: the images are stored in a 64-dimensional (8x8 image maps) feature space and $k$-means is able to identify the 10 different classes that populate this 64-dimensional feature space with high confidence!

## Dimensionality reduction with Principal Components Analysis (PCA)

Let's go back to the Iris data set and see whether we can reduce it to a smaller dimensionality.

In [ ]:
data = load_iris()
x = data.data
y = data.target
x.shape, y.shape

Let's have a look at the pair plot for this data set:

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 10))
iris_plot = sns.load_dataset("iris")
sns.pairplot(iris_plot, diag_kind='hist', hue='species')

PCA is very sensitive to data scaling: the mean of the data set should always be zero, the variance in each feature should be unity. This is not the case here. Therefore, we have to apply scaling with the Standard Scaler, which meets these requirements:

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

Let's apply the PCA:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)

pca.fit(x)

Note that we derive 4 principal components here: this is the maximum number of components. We do not have to use all of them later. By default, all principal components are ordered by importance, the first one being the most important one, the last one being the least important one. Therefore, we can simply extract a slice of the prinicipal components.

Let's have a quick look at the shape of the principal components:

In [ ]:
pca.components_.shape

Of course, this representation is symmetric. Interpreted as a matrix, the rows refer to the different principal components, the columns refer to the original feature vectors based on which the principal components are described.

We can now transform any data point into a representation based on the principal components:

In [ ]:
pca.transform(x[0].reshape(1, -1))

We plan to apply dimensionality. So how many principal components should we keep? Let's have a look at the explained variance for the different components:

In [ ]:
pca.explained_variance_ratio_

Let's plot the results:

In [ ]:
plt.plot(range(1, 5), np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of principal components')
plt.ylabel('Cumulative explained variance')

Interesting. We can reduce this data set to only two principal components and this still enables us to reproduce 95.8% of the variance in the data set. Let's rederive the PCA using only two principal components and plot the transformed data set:

In [ ]:
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x)
x_pca.shape

We plot the transformed data:

In [ ]:
plt.scatter(x_pca[:,0], x_pca[:,1])
plt.xlabel('PCA1')
plt.xlabel('PCA2')

This looks very similar to some of the pair plots that we saw earlier - which makes sense: performing PCA is the equivalent of rotating the data and projecting it to a lower dimensional space.

## Performing PCA on image data

We will now perform PCA to some image data to get a better understanding of what it does.

First we have to download an image data set containing 400 faces of 40 different people. Each sample is a linearized 64x64 image map.

In [ ]:
from sklearn.datasets import fetch_olivetti_faces

data = fetch_olivetti_faces()

x = data.data
y = data.target
x.shape

Let's plot the first 100 images in the data set to get a better idea of the data:

In [ ]:
f, ax = plt.subplots(10, 10, figsize=(10, 10))
ax = np.ravel(ax)

for i in range(100): 
    ax[i].imshow(x[i].reshape(64,64), cmap='Greys_r')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

Ok, so there's 10 images of each person taken from different angles. 

The question now is: can we represent this data set in a lower dimensional space (current dimensionality: 4096=64x64)?

Let's try PCA:

In [ ]:
pca = PCA(n_components=200)
pca.fit(scaler.fit_transform(x))

Let us have a look at the first 10 principal components. Disclaimer: there's some spooky faces...

In [ ]:
f, ax = plt.subplots(2, 5, figsize=(10, 4))
ax = np.ravel(ax)

for i in range(10):
    ax[i].imshow(pca.components_[i].reshape(64, 64), cmap='Greys_r')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

All those principal components look a little bit like faces. Each principal component describes a different set of features in images of faces:
* The first component is a dark face with a bright background.
* The second component is brighter to the right than to the left.
* The third component has dark cheeks.
* The fourth component has well-defined eyes and features of glasses.
* The fifth component has bright regions around the eyes.

... and so on. Each principal components covers different features of faces. This is the **feature selection** part of PCA: the method extracts the most important features.

By transforming an original image based on the PCA transformation, we can reassemble that image based on those principal components. Taking advantage of the **dimensionality reduction** part of PCA, we will only utilize a subset of available principal components: 

In [ ]:
f, ax = plt.subplots(1, 5, figsize=(15 ,3))
ax = np.ravel(ax)

m = 10  # face id

ax[0].imshow(x[m].reshape(64, 64), cmap='Greys_r')
ax[0].set_title('Ground Truth')

for i, j in enumerate([4, 9, 99, 199]):
    ax[i+1].imshow(np.sum(pca.transform(x[m].reshape(1, -1))[0, :j]*pca.components_[:j].transpose(), axis=1).reshape(64,64), cmap='Greys_r')
    ax[i+1].set_title('{} components'.format(j+1))


Naturally, the more principal components we consider, the better is the quality of the reconstruction. The image quality based on 100 principal components is already pretty good. 

Make yourself aware what this means: we can store an image of a person with 4096 pixels by storing only 100 values, if we know the corresponding principal components. Pretty neat, right? 